In [1]:
#Variables
import config

#Preprocessing
import preprocessing_functions as pf

#Libraries
import requests
import json
import pandas as pd
from sqlalchemy import text
from datetime import date

heute = date.today().strftime("%Y-%m-%d")
#Verbindung zur DB aufbauen mit preprocessing function
pf.db_connect(config.db_name)

#Startdatum + Enddatum definieren: Enddatum = heute, Startdatum = gestern-30 Tage
startdatum = config.date_Vor30Tagen
enddatum = config.date_heute

#Daten aus Open-Meteo (Temperatur, Luftfeuchtigkeit etc.)
response = requests.get('https://api.open-meteo.com/v1/forecast?latitude=47.37&longitude=8.54&hourly=temperature_2m,relativehumidity_2m,dewpoint_2m,apparent_temperature,precipitation_probability,precipitation,rain,showers,snowfall,snow_depth,weathercode,pressure_msl,surface_pressure,cloudcover,cloudcover_low,cloudcover_mid,cloudcover_high,visibility,evapotranspiration,et0_fao_evapotranspiration,vapor_pressure_deficit,windspeed_10m,windspeed_80m,windspeed_120m,windspeed_180m,winddirection_10m,winddirection_80m,winddirection_120m,winddirection_180m,windgusts_10m,temperature_80m,temperature_120m,temperature_180m,soil_temperature_0cm,soil_temperature_6cm,soil_temperature_18cm,soil_temperature_54cm,soil_moisture_0_1cm,soil_moisture_1_3cm,soil_moisture_3_9cm,soil_moisture_9_27cm,soil_moisture_27_81cm,uv_index,uv_index_clear_sky,is_day,cape,freezinglevel_height,shortwave_radiation,direct_radiation,diffuse_radiation,direct_normal_irradiance,terrestrial_radiation,shortwave_radiation_instant,direct_radiation_instant,diffuse_radiation_instant,direct_normal_irradiance_instant,terrestrial_radiation_instant,temperature_1000hPa,temperature_975hPa,temperature_950hPa,temperature_925hPa,temperature_900hPa,temperature_850hPa,temperature_800hPa,temperature_700hPa,temperature_600hPa,temperature_500hPa,temperature_400hPa,temperature_300hPa,temperature_250hPa,temperature_200hPa,temperature_150hPa,temperature_100hPa,temperature_70hPa,temperature_50hPa,temperature_30hPa,relativehumidity_1000hPa,relativehumidity_975hPa,relativehumidity_950hPa,relativehumidity_925hPa,relativehumidity_900hPa,relativehumidity_850hPa,relativehumidity_800hPa,relativehumidity_700hPa,relativehumidity_600hPa,relativehumidity_500hPa,relativehumidity_400hPa,relativehumidity_300hPa,relativehumidity_250hPa,relativehumidity_200hPa,relativehumidity_150hPa,relativehumidity_100hPa,relativehumidity_70hPa,relativehumidity_50hPa,relativehumidity_30hPa,cloudcover_1000hPa,cloudcover_975hPa,cloudcover_950hPa,cloudcover_925hPa,cloudcover_900hPa,cloudcover_850hPa,cloudcover_800hPa,cloudcover_700hPa,cloudcover_600hPa,cloudcover_500hPa,cloudcover_400hPa,cloudcover_300hPa,cloudcover_250hPa,cloudcover_200hPa,cloudcover_150hPa,cloudcover_100hPa,cloudcover_70hPa,cloudcover_50hPa,cloudcover_30hPa,windspeed_1000hPa,windspeed_975hPa,windspeed_950hPa,windspeed_925hPa,windspeed_900hPa,windspeed_850hPa,windspeed_800hPa,windspeed_700hPa,windspeed_600hPa,windspeed_500hPa,windspeed_400hPa,windspeed_300hPa,windspeed_250hPa,windspeed_200hPa,windspeed_150hPa,windspeed_100hPa,windspeed_70hPa,windspeed_50hPa,windspeed_30hPa,winddirection_1000hPa,winddirection_975hPa,winddirection_950hPa,winddirection_925hPa,winddirection_900hPa,winddirection_850hPa,winddirection_800hPa,winddirection_700hPa,winddirection_600hPa,winddirection_500hPa,winddirection_400hPa,winddirection_300hPa,winddirection_250hPa,winddirection_200hPa,winddirection_150hPa,winddirection_100hPa,winddirection_70hPa,winddirection_50hPa,winddirection_30hPa&models=best_match,ecmwf_ifs04,metno_nordic,gfs_seamless,jma_seamless,icon_seamless,gem_seamless,meteofrance_seamless&daily=weathercode,temperature_2m_max,temperature_2m_min,apparent_temperature_max,apparent_temperature_min,sunrise,sunset,uv_index_max,uv_index_clear_sky_max,precipitation_sum,rain_sum,showers_sum,snowfall_sum,precipitation_hours,precipitation_probability_max,windspeed_10m_max,windgusts_10m_max,winddirection_10m_dominant,shortwave_radiation_sum,et0_fao_evapotranspiration&start_date=' + startdatum + '&end_date=' + enddatum + '&timezone=Europe%2FBerlin')
weather_data = json.loads(response.content)
daily_data = weather_data['daily']

# DataFrame erstellen:
weather_df = pd.DataFrame(daily_data)

# Datum als Idex
weather_df.set_index('time', inplace=True)

#Daten bereinigen (Entfernung NAs)
weather_df=weather_df.drop(columns=['shortwave_radiation_sum_jma_seamless', 'et0_fao_evapotranspiration_jma_seamless'])

#Daten in Wetter-Table der DB schreiben mit preprocessing Funktion
weather_df.to_sql(config.db_weather, config.db_login, if_exists='replace', index=False)

#--------------------------------------------------------------------------

#Daten aus Air Quality Index laden
from bs4 import BeautifulSoup

URL = "https://www.bafu.admin.ch/bafu/de/home/themen/luft/zustand/daten/luftbelastung--historische-daten/nabel--tageswerte-der-letzten-30--tage/werte-der-letzten-30-tage-nabel--feinstaub--pm10-.html"

f = requests.get(URL)
soup = BeautifulSoup(f.text)

#slicer - Unnötige Informationen entfernen: Alle Wörter vor dem string "Hoch-geb." werden entfernt sowie alle Wörter nach "Legende". Genau in dieser Spanne sind die eigentlichen Daten.
strValue = soup.get_text()

bereinigteListe = strValue.split("Hoch-geb.\n\n\n", 1)
if len(bereinigteListe) > 0:
    strValue=bereinigteListe[1]

strValue = strValue.split("\n\n\n\n\n\n\n     Legende", 1)[0]

#Daten aus Air Quality Index in Tabellen-Format bringen
from io import StringIO

data = strValue.split('\n\n')

data[0] = '\n\n' + data[0]

columns = data[0].split('\n')[1:]

rows = [row[1:] for row in data[1:]]

rows = [row.split('\n') for row in data[1:]]

rows = [row[:len(columns)] for row in rows]

AQI_df = pd.DataFrame(rows, columns=columns)

AQI_df = AQI_df.drop('', axis=1)

#Ganze Table löschen, fals mal wieder was verschoben ist...
#pf.db_drop_table(config.db_name, config.db_AQI_RAW)

#Daten in AQI-Table der DB schreiben mit preprocessing Funktion
AQI_df.to_sql(config.db_AQI_RAW, config.db_login, if_exists='append', index=False)


#Die überflüssigen Daten aus dem AQI DF löschen (Air Quality-Spalten für die anderen Kantone sowie die Datumsspalte) - Hierzu Hinweis: Die Spalten sind irgendwie verrutscht, deshalb muss die Spalte "BAS" behalten werden, sie zeigt aber die Daten für Zürich an.
AQI_df = AQI_df.iloc[1:]

AQI_df = AQI_df[['BAS']]

print(AQI_df)

#--------------------------------------------------------------------

#Umbenennen der Spalte in "AQI-BAFU" (Air Quality Index - Bundesamt für Umwelt)
AQI_df = AQI_df.rename(columns={"BAS": "AQI - BAFU"})

#Index zurücksetzen, damit die beiden Tabellen verbunden werden können:
AQI_df_reset = AQI_df.reset_index(drop=True)
weather_df_reset = weather_df.reset_index(drop=True)

#Tabellen verbinden
all_df = pd.concat([weather_df_reset, AQI_df_reset], axis=1)

#print(all_df)
all_df.to_csv(config.csv_export, index=False)



   BAS
1   11
2    8
3    4
4    5
5    5
6    7
7    5
8    8
9   11
10  13
11  13
12   6
13  10
14   8
15   4
16   5
17   4
18   9
19   9
20   8
21   9
22  10
23   8
24  10
25  13
26  13
27  12
28  11
29   8
30   6


In [2]:
SQLquery = text('SELECT * FROM ' + config.db_weather)
df_METEODB = pd.read_sql(SQLquery, con=config.db_login.connect())

df_METEODB.head()

,weathercode_best_match,temperature_2m_max_best_match,temperature_2m_min_best_match,apparent_temperature_max_best_match,apparent_temperature_min_best_match,sunrise,sunset,uv_index_max_best_match,uv_index_clear_sky_max_best_match,precipitation_sum_best_match,...,apparent_temperature_max_meteofrance_seamless,apparent_temperature_min_meteofrance_seamless,precipitation_sum_meteofrance_seamless,snowfall_sum_meteofrance_seamless,precipitation_hours_meteofrance_seamless,windspeed_10m_max_meteofrance_seamless,windgusts_10m_max_meteofrance_seamless,winddirection_10m_dominant_meteofrance_seamless,shortwave_radiation_sum_meteofrance_seamless,et0_fao_evapotranspiration_meteofrance_seamless
0,3,13.1,5.1,10.3,2.8,2023-04-09T06:47,2023-04-09T20:07,5.45,5.45,0.0,...,10.0,1.2,0.0,0.0,0.0,12.0,29.5,44,24.23,3.37
1,45,16.6,3.9,15.0,1.6,2023-04-10T06:45,2023-04-10T20:08,5.40,5.70,0.0,...,15.3,0.6,0.0,0.0,0.0,7.7,30.6,264,23.03,3.61
2,80,14.1,9.5,11.8,6.7,2023-04-11T06:43,2023-04-11T20:10,3.05,5.40,11.5,...,11.4,6.7,4.1,0.0,7.0,18.5,42.5,247,10.63,2.08
3,61,14.3,8.6,11.7,5.9,2023-04-12T06:41,2023-04-12T20:11,4.65,5.90,8.2,...,13.6,5.2,8.7,0.0,14.0,20.5,46.1,224,11.09,1.96
4,80,8.9,5.2,6.3,2.4,2023-04-13T06:39,2023-04-13T20:12,4.20,5.45,4.8,...,7.1,1.0,4.1,0.0,9.0,20.9,56.2,259,16.27,2.13


In [3]:
SQLquery = text('SELECT * FROM ' + config.db_AQI_RAW)
df_AQIDB = pd.read_sql(SQLquery, con=config.db_login.connect())

df_AQIDB.head()

,Datum,BER,LAU,LUG,ZUE,BAS,DUE,HAE,SIO,MAG,PAY,TAE,BRM,CHA,RIG,DAV,JUN
0,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
1,09.04.23,18,12,20,11,11,13,14,7,16,10,11,10,8,11,4,0
2,10.04.23,13,9,10,8,8,9,9,8,10,8,7,7,6,7,1,1
3,11.04.23,8,5,18,5,4,4,7,11,17,3,4,3,2,2,1,0
4,12.04.23,7,6,23,4,5,3,7,8,21,4,3,2,2,2,2,1
